# Зависимоти

In [ ]:
import tarfile
import json
from core.settings import (ARCHIVE_PATH, DATASET_TRAIN_PATH,
                          DATASET_EVAL_PATH, FUNCTIONS_PATH,
                          DATASETS_FOLDER)
from core.dataprocess import (get_funcs_from_module, 
                              append_functions_in_json,
                              read_portion_data, 
                              simplify_functions)

# Названия файлов

Названия файлов заданы в модуле `settings.py`.

Путь к папке с датасетами
 * `DATASETS_FOLDER`.

Путь к архиву с датасетом
 * `ARCHIVE_PATH`.

Пути к оригинальным датасетам
 * `DATASET_TRAIN_PATH`;
 * `DATASET_EVAL_PATH`.

Путь к извлеченным функциям
 * `FUNCTIONS_PATH`.

# Выполнение

Распаковка архива с датасетами

In [ ]:
with tarfile.open(ARCHIVE_PATH) as tar:
    tar.extractall(DATASETS_FOLDER)

### Порциональное извлечение функций из основного датасета
Из-за большого размера файлов оперативная память не справиться с обработкой, поэтому производится порциональное извлечение функций из датасета.

* batch_size - размер порции (1000 для ОЗУ объемом 4 Гб)

In [ ]:
batch_size = 1000

# train dataset содержит 100 программ (модулей)
# test dataset содержит 50 программ (модулей)
datasets = [
    (DATASET_TRAIN_PATH, 100),
    (DATASET_EVAL_PATH, 50)
]

for path, num_batches in datasets:
    print('|' + ' '*(num_batches//2) + '|')
    print('[', end='')
    pos = 0 # начальная позиция чтения файла
    
    for i in range(num_batches):
        modules, pos = read_portion_data(path, pos, batch_size)
        
        functions = []
        for module in modules:
            module = json.loads(module)
            functions.extend(get_funcs_from_module(module))
        functions = simplify_functions(functions)    
        append_functions_in_json(FUNCTIONS_PATH, functions)
        
        if i % 2 == 0: print('#',end='')
    print('] (1/2)')
print('Completed!')